In [ ]:
import sys
from pathlib import Path

# Add the parent directory of 'microscope_manager' to sys.path
repo_root = Path(__file__).resolve().parent.parent
sys.path.append(str(repo_root))

ModuleNotFoundError: No module named 'microscope_manager'

In [ ]:
# ViventisManager Example Notebook

# --- Imports ---
from microscope_manager.viventis_manager import ViventisManager
import napari
import os

In [ ]:
# --- Set input data path ---
viventis_path = "/path/to/your/viventis/position_folder"  # Update this to your real path


In [ ]:
# --- Initialize manager ---
manager = ViventisManager(path=viventis_path)


In [ ]:
# --- Access data and metadata ---
data = manager.data          # List of Dask arrays (1 per resolution level)
metadata = manager.metadata  # Dict with axes, scales, units, channel names, etc.

In [ ]:
# --- Show metadata summary ---
print("Axes:", metadata["axes"])
print("Size:", metadata["size"][0])
print("Scales:", metadata["scales"])
print("Channels:", metadata["channel_names"])
print("Colors:", metadata["channel_colors"])

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
viewer = manager.visualize()

In [ ]:
# --- Optional: Build pyramid if not already ---
# manager.build_pyramid(num_levels=3, downscale=2)

# --- Optional: Write to OME-Zarr format ---
# output_zarr_path = "/path/to/output/test.zarr"
# manager.write(output_zarr_path)